In [60]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import certifi

os.environ["SSL_CERT_FILE"] = certifi.where()

import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt

import tensorkrowch as tk
device = torch.device('cpu')

# set to float64
torch.set_default_dtype(torch.float64)

# Hyperparameters

# Data
dataset_name = 'mnist'
batch_size = 64
image_size = 15
input_size = image_size ** 2


# # Model
# bond_dim = 50
# init_method = 'unit'
# block_length = 2
# in_dim = 2
# cum_percentage = 0.98

In [61]:
# Dataset
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(image_size, antialias=True),
                               ])

train_dataset = datasets.MNIST(root='data/',
                               train=True,
                               transform=transform,
                               download=True)
test_dataset = datasets.MNIST(root='data/',
                              train=False,
                              transform=transform,
                              download=True)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True)

In [62]:
class MPS_DMRG(tk.models.MPSLayer):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.parameterize(set_param=False, override=True)

        self.out_node.get_axis('input').name = 'output'

        self.block_position = None
        self.block_length = None

    @property
    def block(self):
        if self.block_position is not None:
            return self.mats_env[self.block_position]
        return None

    def merge_block(self, block_position, block_length):
        if block_position + block_length > self.n_features:
            raise ValueError(
                f'Last position of the block ({block_position + block_length}) '
                f'exceeds the range of MPS sites ({self.n_features})')
        elif block_length < 1:
            raise ValueError(
                '`block_length` should be greater than or equal to 1')

        if self.block_position is not None:
            raise ValueError(
                'Cannot create block if there is already a merged block')

        block_nodes = self.mats_env[block_position:(block_position + block_length)]

        block = block_nodes[0]
        for node in block_nodes[1:]:
            block = tk.contract_between_(block, node)
        block = block.parameterize(True)
        block.name = 'block'

        self.block_position = block_position
        self.block_length = block_length
        self._mats_env = self._mats_env[:block_position] + [block] + \
            self._mats_env[(block_position + block_length):]

    def unmerge_block(self, side='right', rank=None, cum_percentage=None):
        block = self.block

        block_nodes = []
        for i in range(self.block_length - 1):
            node1_axes = block.axes[:2]
            node2_axes = block.axes[2:]

            node, block = tk.split_(block,
                                    node1_axes,
                                    node2_axes,
                                    side=side,
                                    rank=rank,
                                    cum_percentage=cum_percentage)
            block.get_axis('split').name = 'left'
            node.get_axis('split').name = 'right'
            node.name = f'mats_env_({self.block_position + i})'

            block_nodes.append(node)

        block.name = f'mats_env_({self.block_position + i + 1})'
        block_nodes.append(block)

        self._mats_env = self._mats_env[:self.block_position] + block_nodes + \
            self._mats_env[(self.block_position + 1):]

        self.block_position = None
        self.block_length = None

    def contract(self):
        result_mats = []
        for node in self.mats_env:
            while any(['input' in name for name in node.axes_names]):
                for axis in node.axes:
                    if 'input' in axis.name:
                        data_node = node.neighbours(axis)
                        node = node @ data_node
                        break
            result_mats.append(node)

        result_mats = [self.left_node] + result_mats + [self.right_node]

        result = result_mats[0]
        for node in result_mats[1:]:
            result @= node

        return result

# Model hyperparameters
embedding_dim = 2
output_dim = 1
bond_dim = 50
init_method = 'unit'
block_length = 2
cum_percentage = 0.98
num_classes = 10
# Initialize network
model_name = 'mps_dmrg'
mps = MPS_DMRG(n_features=input_size,
               in_dim=embedding_dim,
               out_dim=num_classes,
               bond_dim=bond_dim,
               boundary='obc',
               init_method=init_method,
               device=device)

# Important to set data nodes before merging nodes
mps.set_data_nodes()

def embedding(x):
    x = tk.embeddings.unit(x, dim=embedding_dim)
    return x

# def embedding(x):
#     x_bool = x >= 0.5
#     x_ohot = torch.nn.functional.one_hot(x_bool.long(), num_classes=2)
#     return x_ohot.to(torch.get_default_dtype())

# mps.norm()

In [63]:
# Hyperparameters
learning_rate = 1e-3
weight_decay = 1e-8
num_epochs = 100
move_block_epochs = 100

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

# Check accuracy on training & test to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)

            scores = model(embedding(x))
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        accuracy = float(num_correct) / float(num_samples) * 100
    model.train()
    return accuracy

In [ ]:
from tqdm import tqdm

# Train network
block_position = 0
direction = 1
mps.merge_block(block_position, block_length)
mps.trace(torch.zeros(1, input_size, embedding_dim, device=device))
optimizer = optim.Adam(mps.parameters(),
                       lr=learning_rate,
                       weight_decay=weight_decay)

for epoch in range(num_epochs):
    pbar = tqdm(train_loader, total=len(train_loader), desc=f'Epoch {epoch + 1}')
    for batch_idx, (data, targets) in enumerate(pbar):
        # Get data to cuda if possible
        data = data.to(device)
        targets = targets.to(device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # Forward
        scores = mps(embedding(data))
        loss = criterion(scores, targets)

        # # Forward
        # p = mps(embedding(data))
        # log_z = mps.norm(log_scale=True)
        # # loss = (log_z - p.log()).sum()
        # loss  =

        # Backward
        optimizer.zero_grad()
        loss.backward()

        pbar.set_postfix(loss=loss.item())

        # Gradient descent
        optimizer.step()

        if (batch_idx + 1) % move_block_epochs == 0:
            if block_position + direction + block_length > mps.n_features:
                direction *= -1
            if block_position + direction < 0:
                direction *= -1
            if block_length == mps.n_features:
                direction = 0

            if direction >= 0:
                mps.unmerge_block(side='left',
                                  rank=bond_dim,
                                  cum_percentage=cum_percentage)
            else:
                mps.unmerge_block(side='right',
                                  rank=bond_dim,
                                  cum_percentage=cum_percentage)

            block_position += direction
            mps.merge_block(block_position, block_length)
            mps.trace(torch.zeros(1, input_size, embedding_dim, device=device))
            optimizer = optim.Adam(mps.parameters(),
                                   lr=learning_rate,
                                   weight_decay=weight_decay)

    train_acc = check_accuracy(train_loader, mps)
    test_acc = check_accuracy(test_loader, mps)

    print(f'* Epoch {epoch + 1:<3} ({block_position=}, {direction=})=>'
          f' Train. Acc.: {train_acc:.2f},'
          f' Test Acc.: {test_acc:.2f}')

# Reset before saving the model
mps.reset()
torch.save(mps.state_dict(), f'models/{model_name}_{dataset_name}.pt')

/Users/marawangamal/Documents/github/ptn/.venv/lib/python3.10/site-packages/tensorkrowch/components.py:1506: UserWarning: `tensor` is being cropped to fit the shape of node "stack_data_memory" at non-batch edges
  warnings.warn(f'`tensor` is being cropped to fit the shape of '
/Users/marawangamal/Documents/github/ptn/.venv/lib/python3.10/site-packages/tensorkrowch/components.py:1348: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/python_variable_indexing.cpp:312.)
  return tensor[index]
Epoch 1: 100%|██████████| 938/938 [03:17<00:00,  4.75it/s, loss=2.29]


* Epoch 1   (block_position=9, direction=1)=> Train. Acc.: 11.83, Test Acc.: 11.31


Epoch 2: 100%|██████████| 938/938 [03:27<00:00,  4.52it/s, loss=2.28]


* Epoch 2   (block_position=18, direction=1)=> Train. Acc.: 11.99, Test Acc.: 11.83


Epoch 3: 100%|██████████| 938/938 [03:15<00:00,  4.80it/s, loss=2.31]


...

Epoch 47: 100%|██████████| 938/938 [03:17<00:00,  4.75it/s, loss=0.877]


* Epoch 47  (block_position=23, direction=-1)=> Train. Acc.: 69.29, Test Acc.: 58.96


Epoch 48: 100%|██████████| 938/938 [03:10<00:00,  4.92it/s, loss=1.03] 


* Epoch 48  (block_position=14, direction=-1)=> Train. Acc.: 69.34, Test Acc.: 58.73


Epoch 49:  95%|█████████▌| 895/938 [03:05<00:09,  4.70it/s, loss=0.769]

In [48]:
(data, targets)  = next((iter(train_loader)))

In [49]:
data = data.to(device)
data = data.reshape(data.shape[0], -1)
p = mps(embedding(data))
torch.isnan(p.log()).sum()

tensor(24)

IndexError: Node "mats_env_node_(112)" has no axis with name "input"